In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [ ]:
df=pd.read_csv("../input/consumer_complaints/Consumer_Complaints.csv")

In [ ]:
df.head()

In [ ]:
 df.isnull()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')
#yellow lines indicates missing information in the data

In [ ]:
df.shape


In [ ]:
# Remove Missing values
# Add a column encoding the product as an integer

from io import StringIO
col = ['Product', 'Consumer Complaint']
df = df[col]
df = df[pd.notnull(df['Consumer Complaint'])]
df.columns = ['Product', 'Consumer_Complaint']
df['category_id'] = df['Product'].factorize()[0]
category_id_df = df[['Product', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Product']].values)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
df.groupby('Product').Consumer_Complaint.count().plot.bar(ylim=0)
plt.show()

# We see here imbalance of classes
# We want a classifier that gives high prediction accuracy over the majority class,
# while maintaining reasonable accuracy for the minority classes as the majority classes might be of use

In [ ]:
sns.stripplot(x='Product',y='category_id',data=df)

In [ ]:
df = df.sort_values(['category_id'],ascending=False).reset_index(drop=True)
plt.figure(figsize=(20,20))
sns.barplot(x=df["category_id"],y=df["Product"])
plt.xlabel("Product Name",fontsize=25)
plt.ylabel("words",fontsize=25)
plt.title(" rating",fontsize=15)
plt.show()


In [ ]:
sns.countplot(x='category_id',data=df)

In [ ]:
x=df.iloc[:,2:3]

In [ ]:
x

In [ ]:
df['Consumer_Complaint'].unique()

In [ ]:
kmeans=KMeans(3)


In [ ]:
kmeans.fit(x)

In [ ]:
identified_clusters=kmeans.fit_predict(x)
identified_clusters

In [ ]:
data_with_cluster=df.copy()
data_with_cluster['Cluster']=identified_clusters
data_with_cluster

In [ ]:
sns.jointplot(x='category_id',y='',data=df)

In [ ]:
plt.scatter(data_with_clusters[''])

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.Consumer_Complaint).toarray()
labels = df.category_id
features.shape

In [ ]:

# Use 'sklearn.feature_selection.chi2' to find the terms that are the most correlated with each of the products

from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("--> '{}':".format(Product))
  print("  . Most Correlated Unigrams are :\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most Correlated Bigrams are :\n. {}".format('\n. '.join(bigrams[-N:])))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df['Consumer_Complaint'], df['Product'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)